# Imports

# ODE

## Taylor Series

Taylor series are expansions of a function $f(x)$ by some finite distance $dx$ to $f(x+dx)$.
* In essence, the Taylor series provides a means to predict a function value at one point
in terms of the function value and its derivatives at another point.
* In particular, the theorem states that any smooth function can be approximated as a
polynomial:
$$
f(x_{i+1}) = f(x_i)+f^{'}(x_i)h + f^{''}(x_i)\frac{h^2}{2!} + \ldots + f^{n}(x_i)\frac{h^n}{n!}
$$

We can use this to get the value of a function at a given given point given a deffirential equation.

Let the ODE be:
$$
\frac{dy}{dx} = f(x,y) = x - y^2
$$
Given initial conditions $y(0) = 1$ find $y(0.1)$ using the Taylor series expansion.

### Example

## Picard Method

The aim is to solve the ODE using the integration. Given:
$$
\frac{dy}{dx} = f(x,y)
$$
and limit being $y_0$ to $y$.